In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from scipy.optimize import minimize
from scipy.optimize import differential_evolution

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.preprocessing import StandardScaler

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.linear_model import LinearRegression

from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder

import warnings
warnings.filterwarnings('ignore')


## modelagem GLM

In [6]:
%%time
df_mod_mp_feb23jul24_v2 = pd.read_csv("C://Flavio/Projeto Otimizacao/Modelo Renovacao/df_mod_mp_feb23jul24_v2.csv",
                     encoding='latin-1',
                     delimiter='|')

df_mod_mp_feb23jul24_v2.shape

CPU times: total: 6.23 s
Wall time: 6.62 s


(575924, 105)

## Prepara base de modelagem glm

In [4]:
df_mod_filtered=df_mod_mp_feb23jul24_v2[["premio_ofertado","premio_total", 'premio_creserva','premio_demais_cob',"teste_validacao","ind_renovacao","MD_sexo","cod_corr",
                                "Cod_End","MD_sin_5a_cas","MD_tempo_casa_2","MD_idade_veic","MD_faixa_IScas",
                                "anomes_dt_ter" ,"MD_idade","per_com", "RNS_Casco_calc","MD_menor26","UF","Renovacao_ant",
                                "fx_ir_12m","is_casco_ofer","classe_bonus","MD_fator_ajuste","MD_capac","md_categoria",
                                "MD_comb","MD_qtd_assis_cpf", "idade_ult_RNS","canal","MD_tipo_renov",'FABRICA','MD_est_civil',
                                'peso','Grupo_veiculo_new','vigencia',
                                'Risco_PP_mod', 'Risco_PT_mod','Risco_RF_mod','Risco_DM_mod','Risco_AS_mod',
                                'Risco_DC_mod' ,'Risco_DMO_mod']]

df_mod_filtered['DA_emissao'] =0.2349
df_mod_filtered['DA_producao'] =0.2749
df_mod_filtered['risco_dmais_cob'] =df_mod_filtered['premio_demais_cob']*0.2627*1.4
df_mod_filtered['premio_anterior'] =df_mod_filtered['premio_total']
df_mod_filtered['Act_cost']  = df_mod_filtered['Risco_PP_mod'] + df_mod_filtered['Risco_PT_mod'] + df_mod_filtered['Risco_RF_mod']+ df_mod_filtered['Risco_DM_mod']+ df_mod_filtered['Risco_AS_mod'] + df_mod_filtered['Risco_DC_mod'] +df_mod_filtered['Risco_DMO_mod'] + df_mod_filtered['risco_dmais_cob']
df_mod_filtered['Premio_IC100']  = df_mod_filtered['Act_cost']/(1-df_mod_filtered['per_com']/100-df_mod_filtered['DA_producao'] )
df_mod_filtered["premio"] = df_mod_filtered["premio_ofertado"]

In [5]:
df_mod_filtered = df_mod_filtered[df_mod_filtered['per_com'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['FABRICA'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['MD_est_civil'].notna()]
df_mod_filtered = df_mod_filtered[df_mod_filtered['Renovacao_ant'].notna()]
df_mod_filtered = df_mod_filtered.query('premio_total !=0')
df_mod_filtered = df_mod_filtered.query('vigencia == "Anual"')
df_mod_filtered = df_mod_filtered.query('premio_creserva >= 0')
df_mod_filtered = df_mod_filtered[~df_mod_filtered['anomes_dt_ter'].isin([202310,202309,202308,202302,202402])]
df_mod_filtered = df_mod_filtered.query('premio_total >100')
df_mod_filtered = df_mod_filtered.query('premio_ofertado >100')
df_mod_filtered = df_mod_filtered.query('MD_idade !="ERRO"')
df_mod_filtered = df_mod_filtered.query('is_casco_ofer > 0')
df_mod_filtered = df_mod_filtered[(df_mod_filtered['per_com']>0)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered['per_com']<50)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total <=2.3)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.premio_ofertado/df_mod_filtered.premio_total >=0.7)]
df_mod_filtered = df_mod_filtered[(df_mod_filtered.UF != "SP") | (~df_mod_filtered.Grupo_veiculo_new.isin(["Toyota Hilux CD","Toyota Hilux CS","Toyota Hilux SW4"]))]

df_mod_filtered.shape

(175137, 50)

In [8]:
#df_mod_filtered['ind_renovacao'].describe()

In [9]:
#print(df_mod_filtered['classe_bonus'].value_counts(ascending=False))

In [10]:
df_mod=df_mod_filtered.copy()

In [11]:
def categorizar(df):
    
    df['classe_bonus_bin'] = df['classe_bonus'].apply(lambda x: '00'  if x == 0 else 'Demais')   
    
    df['MD_sexo_bin'] = df['MD_sexo'].map({'Masculino': 'Masculino', 
                                 'Feminino': 'Feminino', 
                                  'ERRO': 'Masculino'}).astype(object)
    
    df['MD_sexo_2bin'] = df['MD_sexo'].map({'Masculino': 1, 
                                 'Feminino': 0, 
                                  'ERRO': 1}).astype(int)


    df['fx_ir_12m_bin'] = df['fx_ir_12m'].apply(lambda x: '01_00_20' if x == '01 - 00-<=020' else
                                                          '02_20_40' if x == '02 - 20-<=040' else
                                                          '03_40_60' if x == '03 - 40-<=060' else
                                                          '04_60_80' if x == '04 - 60-<=080' else
                                                          '05_80_100' if x == '05 - 80-<=100' else
                                                          '02_20_40' if x == 'NI'           else 'ERROR')

  
    df['MD_comb_bin'] = df['MD_comb'].apply(lambda x: 'Gasolina' if x == 'Gasolina'  else 'Demais')

    df['canal_bin'] =  df['canal'].apply(lambda x: 'Corretor_Mais' if x == 'Corretor Mais' else 
                                                   'Corretor_Mais' if x == 'CorretorMais'
                                                                   else    'Demais')
   
    df['per_com_bin'] = df['per_com'].apply(lambda x: 'p_1_10' if x < 10 else
                                                      'p_10_35' if x >= 10 else 'ERROR')    



    df['idade_ult_RNS_bin'] = df['idade_ult_RNS'].apply(lambda x:     'Sem_Sinistro' if x == 'Sem Sinistro' else
                                                                      '01_05_anos' if x == '01 ano' else
                                                                      '01_05_anos' if x == '02 anos' else
                                                                      '01_05_anos' if x == '03 anos' else
                                                                      '01_05_anos' if x == '04 anos' else
                                                                      '01_05_anos' if x == '05 anos' else '06_inf')


    df['UF_bin'] = df['UF'].apply(lambda x:     'AL_CE_RS'  if x == 'AL' else
                                                'AL_CE_RS'  if x == 'CE' else
                                                'AL_CE_RS'  if x == 'RS' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'AP' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'DF' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'ES' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PA' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'PB' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'RN' else 
                                                'AP_DF_ES_PA_PB_RN_SC' if x == 'SC' else 
                                                'GO_MA_MG'             if x == 'GO' else 
                                                'GO_MA_MG'             if x == 'MA' else 
                                                'GO_MA_MG'             if x == 'MG' else 
                                                'MT_PR'                if x == 'MT' else
                                                'MT_PR'                if x == 'PR' else 
                                                'RJ'                   if x == 'RJ' else  
                                                'Demais')

    df['md_tipo_renov_ant_bin'] = df['Renovacao_ant'].apply(lambda x:     '1_SeguroNovo'       if x == '1 - Seguro Novo'  else
                                                                          '2_RenovPropria'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                          '2_RenovPropria'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                          '3_RenovCongenere'   if x == '4 - Renov Congenere sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '5 - Renov Congenere com sinistro' else
                                                                          '3_RenovCongenere'   if x == '6 - Renov BB sem sinistro' else
                                                                          '3_RenovCongenere'   if x == '7 - Renov BB com sinistro' else  
                                                                          '2_RenovPropria')
  
    df['MD_tipo_renov_bin'] = df['MD_tipo_renov'].apply(lambda x:          '2_RenovPropria_sem_sin'     if x == '2 - Renov MAPFRE sem sinistro'   else
                                                                           '3_RenovPropria_com_sin'     if x == '3 - Renov MAPFRE com sinistro'   else
                                                                           'Error')
   
     
    df['Cod_End_bin'] = df['Cod_End'].apply(lambda x: '13509'  if x == 13509 else
                                                      '13509'  if x == 50007 else
                                                      '13509'  if x == 50100 else 'Demais')   

    df['MD_capac_bin'] = df['MD_capac'].apply(lambda x: '5'  if x == 5  else 'Demais')  



    df['fator_ajuste_bin'] = df['MD_fator_ajuste'].apply(lambda x: 'f_80_101'    if x < 101 else
                                                                   'f_101_120'   if x >= 101  else 
                                                                   'f_80_101')   
    
    return df

In [12]:
df_mod = categorizar(df_mod)

In [13]:
df_train = df_mod.query('teste_validacao == "Trein"')
df_train.shape

(291023, 64)

In [14]:
#print(df_train['fator_ajuste_bin'].value_counts(ascending=False)) 

In [16]:
df_num = df_train[['premio', 'per_com', 'MD_tempo_casa_2', 'is_casco_ofer','Premio_IC100','premio_anterior']]
df_cat = df_train[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_y = df_train[['ind_renovacao']]
df_cat_glm=pd.get_dummies(df_cat)
df_cat_glm=pd.concat([df_cat_y,df_num, df_cat_glm],axis=1)

In [17]:
#df_cat_glm.columns.to_list()

In [18]:
import statsmodels.formula.api as smf
#Modelo GLM de retenção
formula =  'ind_renovacao ~ 1 + np.log(premio) + np.divide(premio,is_casco_ofer) + np.log(np.divide(premio, Premio_IC100))+ np.log((np.divide(premio, premio_anterior))) + np.log(MD_tempo_casa_2+1) + per_com + per_com_bin_p_1_10 + canal_bin_Corretor_Mais + Cod_End_bin_13509 + fator_ajuste_bin_f_101_120 + classe_bonus_bin_00 + UF_bin_AL_CE_RS + UF_bin_AP_DF_ES_PA_PB_RN_SC + UF_bin_GO_MA_MG + UF_bin_MT_PR + UF_bin_RJ + MD_comb_bin_Gasolina + md_tipo_renov_ant_bin_1_SeguroNovo + md_tipo_renov_ant_bin_3_RenovCongenere + fx_ir_12m_bin_01_00_20 + fx_ir_12m_bin_03_40_60 + fx_ir_12m_bin_04_60_80 + fx_ir_12m_bin_05_80_100 + MD_tipo_renov_bin_3_RenovPropria_com_sin + idade_ult_RNS_bin_01_05_anos + idade_ult_RNS_bin_06_inf ' 
model_glm = smf.logit(formula , data=df_cat_glm).fit()
model_glm.summary()

Optimization terminated successfully.
         Current function value: 0.562651
         Iterations 6


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:          ind_renovacao   No. Observations:               291023
Model:                          Logit   Df Residuals:                   290996
Method:                           MLE   Df Model:                           26
Date:                Sat, 17 Aug 2024   Pseudo R-squ.:                  0.1855
Time:                        10:52:17   Log-Likelihood:            -1.6374e+05
converged:                       True   LL-Null:                   -2.0104e+05
Covariance Type:            nonrobust   LLR p-value:                     0.000
================================================================================================================
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
Intercept                                        3.3458      0.084     39.759      0.000       3.181       3.511
np.log(premio)                                  -0.4638      0.010    -44.612      0.000      -0.484      -0.443
np.divide(premio, is_casco_ofer)                -5.1803      0.220    -23.571      0.000      -5.611      -4.750
np.log(np.divide(premio, Premio_IC100))         -1.0904      0.016    -69.655      0.000      -1.121      -1.060
np.log((np.divide(premio, premio_anterior)))    -1.2646      0.021    -61.392      0.000      -1.305      -1.224
np.log(MD_tempo_casa_2 + 1)                      0.2280      0.012     18.379      0.000       0.204       0.252
per_com                                          0.0137      0.001     15.490      0.000       0.012       0.015
per_com_bin_p_1_10                               0.1566      0.026      6.130      0.000       0.106       0.207
canal_bin_Corretor_Mais                          0.0576      0.011      5.419      0.000       0.037       0.078
Cod_End_bin_13509                               -0.2318      0.015    -15.792      0.000      -0.261      -0.203
fator_ajuste_bin_f_101_120                       0.1600      0.030      5.381      0.000       0.102       0.218
classe_bonus_bin_00                             -0.2332      0.044     -5.286      0.000      -0.320      -0.147
UF_bin_AL_CE_RS                                 -0.2319      0.014    -16.544      0.000      -0.259      -0.204
UF_bin_AP_DF_ES_PA_PB_RN_SC                     -0.2498      0.014    -17.437      0.000      -0.278      -0.222
UF_bin_GO_MA_MG                                 -0.0697      0.015     -4.563      0.000      -0.100      -0.040
UF_bin_MT_PR                                    -0.1227      0.015     -8.341      0.000      -0.152      -0.094
UF_bin_RJ                                        0.2475      0.021     11.713      0.000       0.206       0.289
MD_comb_bin_Gasolina                             0.3917      0.020     19.642      0.000       0.353       0.431
md_tipo_renov_ant_bin_1_SeguroNovo              -0.4536      0.017    -26.046      0.000      -0.488      -0.419
md_tipo_renov_ant_bin_3_RenovCongenere          -0.1999      0.013    -14.812      0.000      -0.226      -0.173
fx_ir_12m_bin_01_00_20                          -0.7509      0.018    -41.872      0.000      -0.786      -0.716
fx_ir_12m_bin_03_40_60                           0.4693      0.010     45.245      0.000       0.449       0.490
fx_ir_12m_bin_04_60_80                           1.1327      0.014     81.310      0.000       1.105       1.160
fx_ir_12m_bin_05_80_100                          2.0154      0.025     79.114      0.000       1.965       2.065
MD_tipo_renov_bin_3_RenovPropria_com_sin        -0.1554      0.022     -7.108      0.000      -0.198      -0.113
idade_ult_RNS_bin_01_05_anos                    -0.5014      0.016    -31.686

## Base Para escorar e otimizar

In [21]:
df_lote_092024 = pd.read_csv("C://Flavio/Projeto Otimizacao/Otimizacao/Table 7 Base Case Total  - teste novo.csv",
                     encoding='latin-1',
                     delimiter='|')

In [22]:
#f_lote_092024.groupby(['Amostra'])['GBM_mercado',"Renewal Demand"].describe().T

In [23]:
#df_lote_092024.columns.to_list()

In [24]:
df_lote_escorar=df_lote_092024.copy()

In [25]:
df_lote_escorar["premio"] = df_lote_escorar["Premio_oferecido_com_batente"]

In [26]:
df_lote_escorar.rename(columns={'Pr_anterior_TI':'premio_anterior', 
                                   'cod_end': 'Cod_End',
                                   'md_capac': 'MD_capac',
                                   'Classe_bonus':'classe_bonus',
                                   'fator_ajuste': 'MD_fator_ajuste'},inplace=True)

In [27]:
df_lote = categorizar(df_lote_escorar)

In [28]:
#df_num = df_train[['premio', 'per_com', 'MD_tempo_casa_2', 'is_casco_ofer','Premio_IC100','premio_anterior']]
df_cat = df_train[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
#df_cat_y = df_train[['ind_renovacao']]
df_cat_glm=pd.get_dummies(df_cat)
df_cat_glm=pd.concat([df_lote_escorar, df_cat_glm],axis=1)

In [29]:
df_lote_escorar.shape

(27304, 138)

In [30]:
df_cat = df_lote[['canal_bin', 'Cod_End_bin', 'fator_ajuste_bin', 'per_com_bin', 'classe_bonus_bin', 'UF_bin', 'MD_comb_bin', 'md_tipo_renov_ant_bin', 'fx_ir_12m_bin', 'MD_tipo_renov_bin','idade_ult_RNS_bin']]
df_cat_glm2=pd.get_dummies(df_cat)
df_lote=pd.concat([df_lote,df_cat_glm2],axis=1)

In [31]:
#df_cat_glm2.columns.to_list()

In [32]:
df_lote["prob_est"]=model_glm2.predict(df_lote)
df_lote["per_com_pct"] = df_lote['per_com']/100
df_lote["premio_prob"] = df_lote["prob_est"]*df_lote["premio"]
df_lote["Act_cost_prob"] = df_lote["prob_est"]*df_lote["Act_cost"]                                                                
df_lote["comissao_valor"] = df_lote["per_com_pct"] *df_lote["premio"]
df_lote["comissao_valor_prob"] = df_lote["per_com_pct"] *df_lote["premio_prob"]


In [33]:
df_lote.shape

(27304, 175)

In [34]:
#df_lote.groupby(["Amostra"])["prob_est" , "Renewal Demand"].describe().T

In [35]:
apolice_ofertada  = np.round(df_lote.shape[0],0)
apolice_estimada  = np.round(np.sum(df_lote["prob_est"]),0)
premio_ofertado   = np.mean(df_lote["premio"])
retencao_estimada = np.mean(df_lote["prob_est"])
premio_esperado   = np.sum(df_lote["premio_prob"])/np.sum(df_lote["prob_est"])
risco_ofertado    = np.mean(df_lote["Act_cost"])
risco_esperado    = np.sum(df_lote["Act_cost_prob"])/np.sum(df_lote["prob_est"])
sp_ofertado       = np.sum(df_lote["Act_cost"])/np.sum(df_lote["premio"])
sp_esperado       = np.sum(df_lote["Act_cost_prob"])/np.sum(df_lote["premio_prob"])
comis_ofertado    = np.sum(df_lote["comissao_valor"])/np.sum(df_lote["premio"])
comis_esperado    = np.sum(df_lote["comissao_valor_prob"])/np.sum(df_lote["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749
print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

Apolices Ofertadas: 27304
Apolices Renovadas: 12832.0
premio_ofertado: 2927.89
retencao_estimada : 47.0
premio_esperado" : 2357.44
risco_ofertado" : 1769.9
risco_esperado" : 1400.96
sp_ofertado" : 60.45
sp_esperado" : 59.43
ic_ofertado" : 103.57
ic_esperado" : 102.57


## Otimização Teste 

In [36]:
#print(df_lote['canal'].value_counts(ascending=False))

In [37]:
#df_lote_otim=df_mod.copy()
df_lote_otim_atu = df_lote.query('Amostra == "Otimização Atu"')
df_lote_otim_atu.shape

(10925, 175)

In [38]:
precos_base=np.array(df_lote_otim_atu['premio'])
indicadora_base=np.array(df_lote_otim_atu['MD_tipo_renov_bin_3_RenovPropria_com_sin'])

# Define a função de demanda
def demanda_base(X,P):
    D = np.exp(0.8164 -0.2974 * X - 0.0003 * P) / (1 + np.exp(0.8164 -0.2974 * X - 0.0003 * P))
    return D

In [39]:
df_lote_otim_atu['Demanda_base']=demanda_base(indicadora_base,precos_base)

In [40]:
df_lote_otim_atu.groupby(["Amostra"])["prob_est" , "Demanda_base"].describe().T

Amostra             Otimização Atu
prob_est     count    10925.000000
             mean         0.472652
             std          0.131939
             min          0.000145
             25%          0.413790
             50%          0.512757
             75%          0.567422
             max          0.673247
Demanda_base count    10925.000000
             mean         0.492113
             std          0.125110
             min          0.000365
             25%          0.440646
             50%          0.530968
             75%          0.580171
             max          0.675214

In [41]:
df_lote_otim_atu.shape

(10925, 176)

In [133]:
df_lote_otim = df_lote_otim_atu

In [42]:
df_lote_otim = df_lote_otim_atu.sample(n =1000) 
df_lote_otim.shape

(1000, 176)

#########################################

In [43]:
df_relatorio_bc = df_lote_otim.copy()

df_relatorio_bc["per_com_pct"] = df_relatorio_bc['per_com']/100
df_relatorio_bc["premio_prob"] = df_relatorio_bc["Demanda_base"]*df_relatorio_bc["premio"]
df_relatorio_bc["Act_cost_prob"] = df_relatorio_bc["Demanda_base"]*df_relatorio_bc["Act_cost"]                                                                
df_relatorio_bc["comissao_valor"] = df_relatorio_bc["per_com_pct"] *df_relatorio_bc["premio"]
df_relatorio_bc["comissao_valor_prob"] = df_relatorio_bc["per_com_pct"] *df_relatorio_bc["premio_prob"]

apolice_ofertada  = np.round(df_relatorio_bc.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio_bc["Demanda_base"]),0)
premio_ofertado   = np.mean(df_relatorio_bc["premio"])
retencao_estimada = np.mean(df_relatorio_bc["Demanda_base"])
premio_esperado   = np.sum(df_relatorio_bc["premio_prob"])/np.sum(df_relatorio_bc["Demanda_base"])
risco_ofertado    = np.mean(df_relatorio_bc["Act_cost"])
risco_esperado    = np.sum(df_relatorio_bc["Act_cost_prob"])/np.sum(df_relatorio_bc["Demanda_base"])
sp_ofertado       = np.sum(df_relatorio_bc["Act_cost"])/np.sum(df_relatorio_bc["premio"])
sp_esperado       = np.sum(df_relatorio_bc["Act_cost_prob"])/np.sum(df_relatorio_bc["premio_prob"])
comis_ofertado    = np.sum(df_relatorio_bc["comissao_valor"])/np.sum(df_relatorio_bc["premio"])
comis_esperado    = np.sum(df_relatorio_bc["comissao_valor_prob"])/np.sum(df_relatorio_bc["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749

print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

Apolices Ofertadas: 1000
Apolices Renovadas: 496.0
premio_ofertado: 2855.89
retencao_estimada : 49.58
premio_esperado" : 2348.33
risco_ofertado" : 1763.4
risco_esperado" : 1419.69
sp_ofertado" : 61.75
sp_esperado" : 60.46
ic_ofertado" : 104.98
ic_esperado" : 103.75


In [194]:
np.mean(df_relatorio_bc['Renewal Demand'])

0.49618017744281706

In [195]:
np.mean(df_relatorio_bc['prob_est'])

0.4732293168059332

In [65]:
np.mean(df_relatorio_bc['Demanda_base'])

0.4957970561324635

In [ ]:
#df2 = df[['premio','is_casco_ofer','Cod_End_bin_Demais','Act_cost']]

In [ ]:
#df2.to_excel(r'analise_teste.xlsx',  header='true')

In [ ]:
#v2=np.mean(df2['is_casco_ofer'])
#indicadora = np.array(df2['Cod_End_bin_Demais']) 
#precos_iniciais = np.array(df2['premio']) 

In [169]:
[x for x in df.columns if "com_bin" in x]

['per_com_bin', 'per_com_bin_p_10_35', 'per_com_bin_p_1_10']

In [57]:
df_lote_otim = df_lote_otim_atu.sample(n =2) 
df_lote_otim.shape

(2, 176)

In [80]:
df = df_lote_otim
P1=np.array(df['premio'])
X11=np.array(df['is_casco_ofer'])
X22=np.array(df['Premio_IC100'])
X33=np.array(df['premio_anterior'])
X44=np.array(df['MD_tempo_casa_2'])
X55=np.array(df['per_com'])
X66=np.array(df['per_com_bin_p_1_10'])
X77=np.array(df['canal_bin_Corretor_Mais'])
X88=np.array(df['Cod_End_bin_13509'])
X99=np.array(df['fator_ajuste_bin_f_101_120'])
X100=np.array(df['classe_bonus_bin_00'])
X110=np.array(df['UF_bin_AL_CE_RS'])
X120=np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
X130=np.array(df['UF_bin_GO_MA_MG'])
X140=np.array(df['UF_bin_MT_PR'])
X150=np.array(df['UF_bin_RJ'])
X160=np.array(df['MD_comb_bin_Gasolina'])
X170=np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
X180=np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
X190=np.array(df['fx_ir_12m_bin_01_00_20'])
X200=np.array(df['fx_ir_12m_bin_03_40_60'])
X210=np.array(df['fx_ir_12m_bin_04_60_80'])
X220=np.array(df['fx_ir_12m_bin_05_80_100'])
X230=np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
X240=np.array(df['idade_ult_RNS_bin_01_05_anos'])
X250=np.array(df['idade_ult_RNS_bin_06_inf'])

# Define a função de demanda
def demanda(P1,X11,X22,X33,X44,X55,X66,X77,X88,X99,X100,X110,X120,X130,X140,X150,X160,X170,X180,X190,X200,X210,X220,X230,X240,X250):
    D = np.exp(3.3458-0.4638 * np.log(P1)-5.1803 * np.divide(P1, X11)-1.0904 * np.log(np.divide(P1, X22))-1.2646 * np.log((np.divide(P1, X33)))  + 0.228 * np.log(X44 + 1)  + 0.0137 * X55  + 0.1566 * X66  + 0.0576 * X77-0.2318 * X88  + 0.16 * X99-0.2332 * X100-0.2319 * X110-0.2498 * X120-0.0697 * X130-0.1227 * X140  + 0.2475 * X150  + 0.3917 * X160-0.4536 * X170-0.1999 * X180-0.7509 * X190  + 0.4693 * X200  + 1.1327 * X210  + 2.0154 * X220-0.1554 * X230-0.5014 * X240-0.4664 * X250)/(1 + np.exp(3.3458-0.4638 * np.log(P1)-5.1803 * np.divide(P1, X11)-1.0904 * np.log(np.divide(P1, X22))-1.2646 * np.log((np.divide(P1, X33)))  + 0.228 * np.log(X44 + 1)  + 0.0137 * X55  + 0.1566 * X66  + 0.0576 * X77-0.2318 * X88  + 0.16 * X99-0.2332 * X100-0.2319 * X110-0.2498 * X120-0.0697 * X130-0.1227 * X140  + 0.2475 * X150  + 0.3917 * X160-0.4536 * X170-0.1999 * X180-0.7509 * X190  + 0.4693 * X200  + 1.1327 * X210  + 2.0154 * X220-0.1554 * X230-0.5014 * X240-0.4664 * X250) )  
    return D

In [81]:
demanda(P1,X11,X22,X33,X44,X55,X66,X77,X88,X99,X100,X110,X120,X130,X140,X150,X160,X170,X180,X190,X200,X210,X220,X230,X240,X250).mean()

0.33126108548552585

In [61]:
df_lote_otim[['premio',
              'is_casco_ofer',
    'Premio_IC100',
    'premio_anterior',
    'MD_tempo_casa_2',
    'per_com',
    'per_com_bin_p_1_10',
    'canal_bin_Corretor_Mais',
    'Cod_End_bin_13509',
    'fator_ajuste_bin_f_101_120',
    'classe_bonus_bin_00',
    'UF_bin_AL_CE_RS',
    'UF_bin_AP_DF_ES_PA_PB_RN_SC',
    'UF_bin_GO_MA_MG',
    'UF_bin_MT_PR',
    'UF_bin_RJ',
    'MD_comb_bin_Gasolina',
    'md_tipo_renov_ant_bin_1_SeguroNovo',
    'md_tipo_renov_ant_bin_3_RenovCongenere',
    'fx_ir_12m_bin_01_00_20',
    'fx_ir_12m_bin_03_40_60',
    'fx_ir_12m_bin_04_60_80',
    'fx_ir_12m_bin_05_80_100',
    'MD_tipo_renov_bin_3_RenovPropria_com_sin',
    'idade_ult_RNS_bin_01_05_anos',
    'idade_ult_RNS_bin_06_inf']]

,premio,is_casco_ofer,Premio_IC100,premio_anterior,MD_tempo_casa_2,per_com,per_com_bin_p_1_10,canal_bin_Corretor_Mais,Cod_End_bin_13509,fator_ajuste_bin_f_101_120,...,MD_comb_bin_Gasolina,md_tipo_renov_ant_bin_1_SeguroNovo,md_tipo_renov_ant_bin_3_RenovCongenere,fx_ir_12m_bin_01_00_20,fx_ir_12m_bin_03_40_60,fx_ir_12m_bin_04_60_80,fx_ir_12m_bin_05_80_100,MD_tipo_renov_bin_3_RenovPropria_com_sin,idade_ult_RNS_bin_01_05_anos,idade_ult_RNS_bin_06_inf
3438,1227.291969,22139.0,1227.291969,1270.24,3,27.0,0,1,0,0,...,0,0,0,0,1,0,0,0,0,0
13770,2015.561125,89652.0,2015.561125,2258.33,1,10.0,0,0,1,0,...,1,0,1,0,1,0,0,0,0,0


In [71]:
%%time
######
import numpy as np
import sympy as sp
from scipy.optimize import fsolve

#df = df_lote_otim

precos_iniciais=np.array(df['premio'])
P=(df['premio'])


lista_variaveis_na_ordem = ['is_casco_ofer',
    'Premio_IC100',
    'premio_anterior',
    'MD_tempo_casa_2',
    'per_com',
    'per_com_bin_p_1_10',
    'canal_bin_Corretor_Mais',
    'Cod_End_bin_13509',
    'fator_ajuste_bin_f_101_120',
    'classe_bonus_bin_00',
    'UF_bin_AL_CE_RS',
    'UF_bin_AP_DF_ES_PA_PB_RN_SC',
    'UF_bin_GO_MA_MG',
    'UF_bin_MT_PR',
    'UF_bin_RJ',
    'MD_comb_bin_Gasolina',
    'md_tipo_renov_ant_bin_1_SeguroNovo',
    'md_tipo_renov_ant_bin_3_RenovCongenere',
    'fx_ir_12m_bin_01_00_20',
    'fx_ir_12m_bin_03_40_60',
    'fx_ir_12m_bin_04_60_80',
    'fx_ir_12m_bin_05_80_100',
    'MD_tipo_renov_bin_3_RenovPropria_com_sin',
    'idade_ult_RNS_bin_01_05_anos',
    'idade_ult_RNS_bin_06_inf']

indicadora=np.array(df.loc[:,lista_variaveis_na_ordem])


target_demand=0.662522170971972

custo=df['Act_cost'].mean()


# Define a função de demanda
def demanda(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25):
    D = np.exp(3.3458-0.4638 * np.log(P)-5.1803 * np.divide(P, X1)-1.0904 * np.log(np.divide(P, X2))-1.2646 * np.log((np.divide(P, X3)))  + 0.228 * np.log(X4 + 1)  + 0.0137 * X5  + 0.1566 * X6  + 0.0576 * X7-0.2318 * X8  + 0.16 * X9-0.2332 * X10-0.2319 * X11-0.2498 * X12-0.0697 * X13-0.1227 * X14  + 0.2475 * X14  + 0.3917 * X16-0.4536 * X17-0.1999 * X18-0.7509 * X19  + 0.4693 * X20  + 1.1327 * X21  + 2.0154 * X22-0.1554 * X23-0.5014 * X24-0.4664 * X25)/(1 + np.exp(3.3458-0.4638 * np.log(P)-5.1803 * np.divide(P, X1)-1.0904 * np.log(np.divide(P, X2))-1.2646 * np.log((np.divide(P, X3)))  + 0.228 * np.log(X4 + 1)  + 0.0137 * X5  + 0.1566 * X6  + 0.0576 * X7-0.2318 * X8  + 0.16 * X9-0.2332 * X10-0.2319 * X11-0.2498 * X12-0.0697 * X13-0.1227 * X14  + 0.2475 * X14  + 0.3917 * X16-0.4536 * X17-0.1999 * X18-0.7509 * X19  + 0.4693 * X20  + 1.1327 * X21  + 2.0154 * X22-0.1554 * X23-0.5014 * X24-0.4664 * X25) )  
    return D

# Define a função de Lagrange
def func(X, indicadora):
    P = X[:-1]  # Extraindo o vetor de preços [p1, p2, ..., pn]
    L = X[-1]   # Extraindo o multiplicador de Lagrange
    n = len(P)
    
    # Função objetivo e a soma das demandas
    objetivo = 0
    soma_demandas = 0
    
    for i in range(n):
        D_i = demanda(P[i],
                     indicadora[i, 0], 
                     indicadora[i, 1],
                     indicadora[i, 2],
                     indicadora[i, 3],
                     indicadora[i, 4],
                     indicadora[i, 5],
                     indicadora[i, 6],
                     indicadora[i, 7],
                     indicadora[i, 8],
                     indicadora[i, 9],
                     indicadora[i, 10],
                     indicadora[i, 11],
                     indicadora[i, 12],
                     indicadora[i, 13],
                     indicadora[i, 14],
                     indicadora[i, 15],
                     indicadora[i, 16],
                     indicadora[i, 17],
                     indicadora[i, 18],
                     indicadora[i, 19],
                     indicadora[i, 20],
                     indicadora[i, 21],
                     indicadora[i, 22],
                     indicadora[i, 23],
                     indicadora[i, 24])
        objetivo += D_i * (P[i] - custo)
        soma_demandas += D_i
    
    # Definindo a função de Lagrange com a restrição
    return objetivo - L * (soma_demandas - target_demand)

# Aplicando a diferenciação numérica
def dfunc(X, indicadora):
    dLambda = np.zeros(len(X))
    h = 1e-2  # Passo usado na diferença finita
    for i in range(len(X)):
        dX = np.zeros(len(X))
        dX[i] = h
        dLambda[i] = (func(X + dX, indicadora) - func(X - dX, indicadora)) / (2 * h)
    return dLambda

# Definindo a função para otimizar os preços
def otimizar_precos(indicadora, precos_iniciais):
    # Iniciando o vetor X com os preços iniciais e um valor inicial para o multiplicador de Lagrange
    X_inicial = np.append(precos_iniciais, 0)
    
    # Calculando os preços otimizados
    X_otimizado = fsolve(dfunc, X_inicial, args=(indicadora))
    
    # Retornando o vetor de preços otimizados e o multiplicador de Lagrange
    precos_otimizados = X_otimizado[:-1]
    lambida = X_otimizado[-1]
    
    return precos_otimizados, lambida


precos_otimizados, lambida = otimizar_precos(indicadora, precos_iniciais)
print("λ:", lambida)
print("Preços Otimizados:",precos_otimizados)



λ: 352.2354049550005
Preços Otimizados: [2144.05182239 2825.14934155]
CPU times: total: 46.9 ms
Wall time: 21.2 ms


In [72]:
precos_iniciais

array([1227.291969, 2015.561125])

In [75]:
indicadora

array([[2.21390000e+04, 1.22729197e+03, 1.27024000e+03, 3.00000000e+00,
        2.70000000e+01, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 1.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00],
       [8.96520000e+04, 2.01556113e+03, 2.25833000e+03, 1.00000000e+00,
        1.00000000e+01, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 1.00000000e+00, 0.00000000e+00, 1.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00]])

In [73]:
X1=np.array(df['is_casco_ofer'])
X2=np.array(df['Premio_IC100'])
X3=np.array(df['premio_anterior'])
X4=np.array(df['MD_tempo_casa_2'])
X5=np.array(df['per_com'])
X6=np.array(df['per_com_bin_p_1_10'])
X7=np.array(df['canal_bin_Corretor_Mais'])
X8=np.array(df['Cod_End_bin_13509'])
X9=np.array(df['fator_ajuste_bin_f_101_120'])
X10=np.array(df['classe_bonus_bin_00'])
X11=np.array(df['UF_bin_AL_CE_RS'])
X12=np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
X13=np.array(df['UF_bin_GO_MA_MG'])
X14=np.array(df['UF_bin_MT_PR'])
X15=np.array(df['UF_bin_RJ'])
X16=np.array(df['MD_comb_bin_Gasolina'])
X17=np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
X18=np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
X19=np.array(df['fx_ir_12m_bin_01_00_20'])
X20=np.array(df['fx_ir_12m_bin_03_40_60'])
X21=np.array(df['fx_ir_12m_bin_04_60_80'])
X22=np.array(df['fx_ir_12m_bin_05_80_100'])
X23=np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
X24=np.array(df['idade_ult_RNS_bin_01_05_anos'])
X25=np.array(df['idade_ult_RNS_bin_06_inf'])



In [76]:
X2

array([22139., 89652.])

In [74]:

# Verificar os preços otimizados e a margem esperada otimizada 
df['precos_otimizados'] = precos_otimizados
df['retencao_otimizada'] = demanda(precos_otimizados,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25)
df['retencao_base_esperado']=demanda(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25)
df['margem_esperada_otimizada'] =   df['retencao_otimizada']*(df['precos_otimizados'] - df['Act_cost'])

df['margem_esperada_base_case'] = df['retencao_base_esperado']*(df['premio']-df['Act_cost'])

margem_esperada_total_base_case=df['margem_esperada_base_case'].sum()
margem_esperada_total_otimizada=df['margem_esperada_otimizada'].sum()

# Exibir os resultados
print("Margem Esperada Total Base Case:", margem_esperada_total_base_case) 
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada) 
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())
print("Retenção Média Base Case:", df['retencao_base_esperado'].mean())
print("Premio Medio Otimizado:", df['precos_otimizados'].mean())
print("Premio Medio Base Case:", df['premio'].mean())
print("Tamanho da Base:", df.shape[0])

Margem Esperada Total Base Case: 941.5945212917563
Margem Esperada Total Otimizada: 1042.439787362356
Retenção Média Otimizada: 0.33126108548443134
Retenção Média Base Case: 0.662522170971972
Premio Medio Otimizado: 2484.600581973951
Premio Medio Base Case: 1621.426547
Tamanho da Base: 2


In [ ]:
#df[['premio','retencao_base_esperado','preco_otimizado','retencao_otimizada']]

In [ ]:
#df#.to_csv(r'C:/Users/vcatelan/Documents/Scripts/dftocsv2.csv', sep='|', encoding='utf-8', header='true')
#df.to_excel(r'df.xlsx', sep='|', encoding='utf-8', header='true')

#criando a função

In [52]:
df_relatorio_otim = df.copy()

df_relatorio_otim["per_com_pct"] = df_relatorio_otim['per_com']/100
df_relatorio_otim["premio_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["Act_cost_prob"] = df_relatorio_otim["retencao_otimizada"]*df_relatorio_otim["Act_cost"]                                                                
df_relatorio_otim["comissao_valor"] = df_relatorio_otim["per_com_pct"] *df_relatorio_otim["precos_otimizados"]
df_relatorio_otim["comissao_valor_prob"] = df_relatorio_otim["per_com_pct"] *df_relatorio_otim["premio_prob"]

apolice_ofertada  = np.round(df_relatorio_otim.shape[0],0)
apolice_estimada  = np.round(np.sum(df_relatorio_otim["retencao_otimizada"]),0)
premio_ofertado   = np.mean(df_relatorio_otim["precos_otimizados"])
retencao_estimada = np.mean(df_relatorio_otim["retencao_otimizada"])
premio_esperado   = np.sum(df_relatorio_otim["premio_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
risco_ofertado    = np.mean(df_relatorio_otim["Act_cost"])
risco_esperado    = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["retencao_otimizada"])
sp_ofertado       = np.sum(df_relatorio_otim["Act_cost"])/np.sum(df_relatorio_otim["precos_otimizados"])
sp_esperado       = np.sum(df_relatorio_otim["Act_cost_prob"])/np.sum(df_relatorio_otim["premio_prob"])
comis_ofertado    = np.sum(df_relatorio_otim["comissao_valor"])/np.sum(df_relatorio_otim["precos_otimizados"])
comis_esperado    = np.sum(df_relatorio_otim["comissao_valor_prob"])/np.sum(df_relatorio_otim["premio_prob"])
ic_ofertado       =  sp_ofertado  +  comis_ofertado + 0.2749
ic_esperado       =  sp_esperado  +  comis_esperado + 0.2749

print('Apolices Ofertadas: %s' % apolice_ofertada)
print('Apolices Renovadas: %s' % apolice_estimada)
print('premio_ofertado: %s' % np.round(premio_ofertado,2))
print('retencao_estimada : %s' % np.round(retencao_estimada*100,2))
print('premio_esperado" : %s'  %np.round( premio_esperado,2))
print('risco_ofertado" : %s'  %  np.round(risco_ofertado,2))
print('risco_esperado" : %s'  %  np.round(risco_esperado,2))
print('sp_ofertado" : %s'  % np.round(sp_ofertado*100,2))
print('sp_esperado" : %s'  % np.round(sp_esperado*100,2))
print('ic_ofertado" : %s'  % np.round(ic_ofertado*100,2))
print('ic_esperado" : %s'  % np.round(ic_esperado*100,2))

Apolices Ofertadas: 10
Apolices Renovadas: 0.0
premio_ofertado: 20922.36
retencao_estimada : 4.96
premio_esperado" : 6780.16
risco_ofertado" : 1920.92
risco_esperado" : 1755.47
sp_ofertado" : 9.18
sp_esperado" : 25.89
ic_ofertado" : 57.02
ic_esperado" : 71.08


In [135]:
df.to_csv(r'df_total.csv', sep='|' ,header='true')

In [ ]:
df_relatorio_otim[['Demanda_base','retencao_otimizada','premio','preco_otimizado']]


In [ ]:
prices_df = pd.DataFrame({
    'chave': df_relatorio["chave"],
    'Act_cost':  df_relatorio["Act_cost"],
    'Premio ofertado com batente':  df_relatorio["Premio_oferecido_com_batente"],
    'Optimized Prices': df_relatorio["premio"],
    'lb_bound' : [b[0] for b in bounds],
    'up_bound' : [b[1] for b in bounds],
    'prob_est':         df_relatorio["prob_est"],
    'renewal demand':df_relatorio["demanda otimizada"],
    'variação':df_relatorio["premio"]/df_relatorio["Premio_oferecido_com_batente"]
})
prices_df.head(20)

########

In [167]:
%%time
target_demand=0.48

df = df_lote_otim
P=(df['premio'])
X1=np.array(df['is_casco_ofer'])
X2=np.array(df['Premio_IC100'])
X3=np.array(df['premio_anterior'])
X4=np.array(df['MD_tempo_casa_2'])
X5=np.array(df['per_com'])
X6=np.array(df['per_com_bin_p_1_10'])
X7=np.array(df['canal_bin_Corretor_Mais'])
X8=np.array(df['Cod_End_bin_13509'])
X9=np.array(df['fator_ajuste_bin_f_101_120'])
X10=np.array(df['classe_bonus_bin_00'])
X11=np.array(df['UF_bin_AL_CE_RS'])
X12=np.array(df['UF_bin_AP_DF_ES_PA_PB_RN_SC'])
X13=np.array(df['UF_bin_GO_MA_MG'])
X14=np.array(df['UF_bin_MT_PR'])
X15=np.array(df['UF_bin_RJ'])
X16=np.array(df['MD_comb_bin_Gasolina'])
X17=np.array(df['md_tipo_renov_ant_bin_1_SeguroNovo'])
X18=np.array(df['md_tipo_renov_ant_bin_3_RenovCongenere'])
X19=np.array(df['fx_ir_12m_bin_01_00_20'])
X20=np.array(df['fx_ir_12m_bin_03_40_60'])
X21=np.array(df['fx_ir_12m_bin_04_60_80'])
X22=np.array(df['fx_ir_12m_bin_05_80_100'])
X23=np.array(df['MD_tipo_renov_bin_3_RenovPropria_com_sin'])
X24=np.array(df['idade_ult_RNS_bin_01_05_anos'])
X25=np.array(df['idade_ult_RNS_bin_06_inf'])


# Função de demanda para calcular a retenção com base no preço 
def calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25):
    D = np.exp(3.3458-0.4638 * np.log(P)-5.1803 * np.divide(P, X1)-1.0904 * np.log(np.divide(P, X2))-1.2646 * np.log((np.divide(P, X3)))  + 0.228 * np.log(X4 + 1)  + 0.0137 * X5  + 0.1566 * X6  + 0.0576 * X7-0.2318 * X8  + 0.16 * X9-0.2332 * X10-0.2319 * X11-0.2498 * X12-0.0697 * X13-0.1227 * X14  + 0.2475 * X14  + 0.3917 * X16-0.4536 * X17-0.1999 * X18-0.7509 * X19  + 0.4693 * X20  + 1.1327 * X21  + 2.0154 * X22-0.1554 * X23-0.5014 * X24-0.4664 * X25)/(1 + np.exp(3.3458-0.4638 * np.log(P)-5.1803 * np.divide(P, X1)-1.0904 * np.log(np.divide(P, X2))-1.2646 * np.log((np.divide(P, X3)))  + 0.228 * np.log(X4 + 1)  + 0.0137 * X5  + 0.1566 * X6  + 0.0576 * X7-0.2318 * X8  + 0.16 * X9-0.2332 * X10-0.2319 * X11-0.2498 * X12-0.0697 * X13-0.1227 * X14  + 0.2475 * X14  + 0.3917 * X16-0.4536 * X17-0.1999 * X18-0.7509 * X19  + 0.4693 * X20  + 1.1327 * X21  + 2.0154 * X22-0.1554 * X23-0.5014 * X24-0.4664 * X25) )  
    return D

df['retencao_base_esperado']=calcular_retencao(P,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25)

df['retencao_target']=target_demand

fator_retencao = np.mean(df['retencao_base_esperado'])/np.mean(df['retencao_target'])

# Definir a função objetivo que maximiza a margem esperada otimizada 
def objetivo(precos):
    df['precos']=precos
    # Calcular a retenção otimizada com os preços dados
    df['retencao_otimizada'] = calcular_retencao(precos,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25)
    df['margem_esperada_otimizada'] =   df['retencao_otimizada']*(df['precos'] - df['Act_cost'])
    
    # Calcular a margem esperada total (somatório)
    margem_esperada_total = df['margem_esperada_otimizada'].sum()

    # Maximizar a margem (retornamos o negativo para que o método minimize possa maximizar)
    return -margem_esperada_total

# Definir a restrição: a retenção média deve ser igual à retenção base média 
def restricao(precos):
    df['precos']=precos
    
    # Calcular a retenção otimizada média com os preços dados
    retencao_otimizada_soma = calcular_retencao(precos,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25).sum()
    
    return np.round(retencao_otimizada_soma,3) - np.round(df['retencao_target'].sum,3)

# Estimativas iniciais (os preços base)

#precos_iniciais = fator_retencao*df['premio'].values

premio_base = df['premio'].values

precos_iniciais = 1*df['premio'].values

# Definir os limites para os preços (0.9 * preco_base <= preco_otimizado <= 1.1 * preco_base) 
limites = [(0.9 * p, 1.1 * p) for p in premio_base]

# Definir as restrições no formato exigido pelo método 'minimize'
restricoes = ({'type': 'eq', 'fun': restricao})

# Executar a otimização 

solution = minimize(objetivo,
                    precos_iniciais,
                    method='L-BFGS-B',
                    bounds=limites,
                    constraints=restricoes) 
# Resultados
precos_otimizados = solution.x
margem_esperada_total_otimizada = -resultado.fun

# Verificar os preços otimizados e a margem esperada otimizada 
df['preco_otimizado'] = solution.x 
df['retencao_otimizada'] = calcular_retencao(precos_otimizados,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25)

df['margem_esperada_base_case'] = df['retencao_base_esperado']*(df['premio']-df['Act_cost'])

margem_esperada_total_base_case=df['margem_esperada_base_case'].sum()

# Exibir os resultados
print(solution.message) 
print("Margem Esperada Total Base Case:", margem_esperada_total_base_case) 
print("Margem Esperada Total Otimizada:", margem_esperada_total_otimizada) 
print("Retenção Média Otimizada:", df['retencao_otimizada'].mean())
print("Retenção Média Base Case:", df['retencao_base_esperado'].mean())
print("Premio Medio Otimizado:", df['preco_otimizado'].mean())
print("Premio Medio Base Case:", df['premio'].mean())
print("Tamanho da Base:", df.shape[0])


STOP: TOTAL NO. of f AND g EVALUATIONS EXCEEDS LIMIT
Margem Esperada Total Base Case: 2472131.8046860155
Margem Esperada Total Otimizada: 748631.2643697124
Retenção Média Otimizada: 0.5134101292779922
Retenção Média Base Case: 0.517242527363945
Premio Medio Otimizado: 2908.7971447498785
Premio Medio Base Case: 2896.3610072528
Tamanho da Base: 5000
CPU times: total: 46.7 s
Wall time: 51.7 s


In [84]:
from handythread import foreach

ModuleNotFoundError: No module named 'handythread'

In [83]:
!pip install pp

Defaulting to user installation because normal site-packages is not writeable
Could not fetch URL https://pypi.org/simple/pp/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pp/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:997)'))) - skipping
Could not fetch URL https://pypi.org/simple/pip/: There was a problem confirming the ssl certificate: HTTPSConnectionPool(host='pypi.org', port=443): Max retries exceeded with url: /simple/pip/ (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:997)'))) - skipping


ERROR: Could not find a version that satisfies the requirement pp (from versions: none)
ERROR: No matching distribution found for pp
